In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import  SubsetRandomSampler 
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import cv2
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA is available!  Training on GPU ...


In [1]:
from tensorflow import keras
from keras.datasets import fashion_mnist

Using TensorFlow backend.


In [5]:
# 3,1
# Create CNN Model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()

        # Convolution 1 , input_shape=(1,28,28)  #out_channel==幾個filter=幾個feature map
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0) #output_shape=(16,26,26)
        self.relu1 = nn.ReLU() # activation
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,ceil_mode=True) #output_shape=(16,13,13)
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0) #output_shape=(32,11,11)
        self.relu2 = nn.ReLU() # activation
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,ceil_mode=True) #output_shape=(32,6,6)
        # Fully connected 1 ,#input_shape=(32*6*6)
        self.fc1 = nn.Linear(32 * 6 * 6, 10) 
      
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Max pool 2 
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        # Linear function (readout)
        out = self.fc1(out)
        return out 

In [6]:
def fit_model(model, criterion, optimizer,epochs,train_loader,test_loader):
    valid_loss_min = np.Inf
    steps = 0
    model.train() #prep model for training
    train_losses, valid_losses = [],[]
    training_accuracy, validation_accuracy = [],[]


    for e in range(epochs):
        running_loss =0 
        valid_loss = 0

        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images=images.to(device)
            labels=labels.to(device)
            optimizer.zero_grad()
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()*images.size(0)


            predicted = torch.max(log_ps.data, 1)[1]
            total_train += len(labels)
            correct_train += (predicted == labels).float().sum()

        correct_test = 0
        total_test = 0

        for images, labels in valid_loader:
            images=images.to(device)
            labels=labels.to(device)
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            valid_loss += loss.item()*images.size(0)

            predicted = torch.max(log_ps.data, 1)[1]
            total_test += len(labels)  
            correct_test += (predicted == labels).float().sum()

        train_accuracy = 1 * correct_train / float(total_train)
        val_accuracy = 1 * correct_test / float(total_test)
        running_loss = running_loss/len(train_loader.sampler)
        valid_loss =  valid_loss/len(valid_loader.sampler)

        train_losses.append(running_loss)
        valid_losses.append(valid_loss)
        training_accuracy.append(train_accuracy)
        validation_accuracy.append(val_accuracy)

        print('Epoch: {} \tTraining Loss: {:.6f} Validation Loss: {:.6f} Training acc: {:.6f} Validation acc: {:.6f}'.format(
            e+1,
            running_loss,
            valid_loss,
            train_accuracy,
            val_accuracy
        ))
    return train_losses, training_accuracy, valid_losses, validation_accuracy       

In [10]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}'.format(test_loss))

    print('Test Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# vgg16

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
model.eval()
model.features[0] = nn.Conv2d(1,64,3,1,1)
model=model.cuda()
print(model)

In [ ]:
from torchvision import models
from torchsummary import summary
summary(model, (1, 32, 32))

# Resnet 18

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model=model.cuda()